In [1]:
import keras
import numpy as np
import os
import sys
import tensorflow as tf
import pandas as pd

from datetime import datetime

sys.path.append('.')

from helpers import iou
from FrameLoader import FrameLoader
from DataGeneratorFrames import DataGeneratorFrames
from DataRepository import DataRepository

from models.GoogleNet import get_model as get_model_googlenet
from models.MobileNetV3Small import get_model as get_model_mobilenet
from models.RandomCNN import get_model as get_model_randomcnn
from models.vitransformer_enc import get_model as get_model_vit

def train_model(model: keras.Sequential, info_train, from_scratch=True):
    """Returns history object"""
    DIM = selected_info['dim']
    weight_path = f"../weights/{selected_info['name']}.weights.h5"
    if not from_scratch and os.path.exists(weight_path):
        model.load_weights(weight_path)

    repo = DataRepository()
    train_generator = DataGeneratorFrames(
        frameloader=FrameLoader(repo),
        train_test_val="train",
        dim=(DIM,DIM),
        batch_size=info_train['batch_size'],
    )
    val_generator = DataGeneratorFrames(
        frameloader=FrameLoader(repo),
        train_test_val="test",
        dim=(DIM,DIM),
        batch_size=info_train['batch_size'],
    )
    
    callbacks = [
        keras.callbacks.ModelCheckpoint('weights/last_trained_model_best.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=1),
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
    ]
    if info_train["early_stopping"]:
        callbacks.append(keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1))
    
    optimizer = keras.optimizers.Adam(learning_rate=info_train['learning_rate'])
    model.compile(optimizer=optimizer, loss='mse', metrics=[iou])

    history = model.fit(
        train_generator,
        epochs=info_train['epochs'],
        callbacks=callbacks,
        verbose=1,
        validation_data=val_generator
    )

    if 'unfreeze_pre_trained_layers_after_training' in info_train.keys():
        pass
        # TODO : save history, add them after next training round

    df_history = pd.DataFrame(history.history)
    df_history["modelname"] = selected_info['name']
    df_history["train_date"] = info_train['train_date']
    last_epoch_nr = int(repo.get_last_epoch_nr(selected_info['name']))
    print("last_epoch", last_epoch_nr, from_scratch)
    if last_epoch_nr > 0:
        # Return when training from scratch has worse results than last time
        # TODO : make a function to update val_iou based on last validation set
        last_result = repo.get_last_epoch_values(modelname=selected_info["name"], epoch=last_epoch_nr)
        print('result now', df_history.loc[df_history.index[-1], 'val_iou'])
        print('last result was: ', last_result.loc[0, 'val_iou'])
        if df_history.loc[df_history.index[-1], 'val_iou'] < last_result.loc[0, 'val_iou']:
            print("RESULTS WEREN'T BETTER")
            return df_history

        df_history["epoch"] = df_history.index + 1 + (0 if from_scratch else last_epoch_nr)
    else:
        df_history["epoch"] = df_history.index + 1
    
    model.save_weights(weight_path)
    repo.save_train_results(df_history)

    return df_history

###############################################################################

info_googlenet = {
    'name' : 'googlenet',
    'dim' : 512,
    'batch_size' : 8,
    'learning_rate' : 1e-4,
    'use_batch_norm' : True,
    'get_model_function' : get_model_googlenet,
}
info_vit = {
    'name' : 'vision_transformer',
    'dim' : 224,
    'patch_size' : 16, # (224 / 16) **2 = 196 patches
    'dim_embedding' : 64,
    'num_heads': 4,
    'encoder_blocks': 4,
    'mlp_head_units' : [1024, 256, 64],  # Size of the dense layers
    'batch_size' : 8,
    'min_epochs' : 15,
    'learning_rate' : 1e-3,
    'weight_decay' : 1e-4,
    'get_model_function' : get_model_vit,
}
info_mobilenet = {
    'name' : 'mobilenet',
    'dim' : 224, # pre-trained default
    'batch_size' : 32,
    'min_epochs' : 15,
    'has_frozen_layers' : True,
    'learning_rate' : 1e-3,
    'get_model_function' : get_model_mobilenet,
}

###############################################################################
selected_info = info_mobilenet
# TODO : continue training
# TODO : save model
# TODO : write results
###############################################################################

trainings_info = {
    'epochs' : 3,
    'early_stopping' : False,
    'early_stopping_patience' : 3,
    'batch_size' : selected_info['batch_size'],
    'learning_rate' : 1e-4 if 'learning_rate' not in selected_info.keys() else selected_info['learning_rate'],
    'train_date' : datetime.now().strftime("%Y%m%d"),
}
trainings_info['weight_decay'] = trainings_info['learning_rate'] / 10 if 'weight_decay' not in selected_info.keys() else selected_info['weight_decay']

model = selected_info['get_model_function'](selected_info)
model.summary()

history = train_model(model, info_train=trainings_info, from_scratch=False)

print(history)


2025-01-17 15:41:18.748354: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-17 15:41:18.759214: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737124878.772384  249411 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737124878.776023  249411 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-17 15:41:18.789584: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ MobileNetV3Small (Functional)   │ (None, 576)            │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       295,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,300,724 (4.96 MB)

 Trainable params: 361,604 (1.38 MB)

 Non-trainable params: 939,120 (3.58 MB)

connection established, loading frame paths
rel paths of framelabels loaded
connection established, loading frame paths
rel paths of framelabels loaded
DataGeneratorSkillBorders init done
connection established, loading frame paths
rel paths of framelabels loaded
DataGeneratorSkillBorders init done
Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - iou: 0.1419 - loss: 0.0554
Epoch 1: val_loss improved from inf to 0.01345, saving model to weights/last_trained_model_best.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - iou: 0.1419 - loss: 0.0554 - val_iou: 0.2854 - val_loss: 0.0135 - learning_rate: 0.0010
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - iou: 0.2597 - loss: 0.0148
Epoch 2: val_loss improved from 0.01345 to 0.00788, saving model to weights/last_trained_model_best.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - iou: 0.2597 - loss: 0.0148 - val_iou: 0.2201 - val_loss: 0.0079 - learning_rate: 0.0010
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - iou: 0.1509 - loss: 0.0091
Epoch 3: va

In [2]:
history

,iou,loss,val_iou,val_loss,learning_rate,modelname,train_date
0,0.141935,0.055430,0.285442,0.013452,0.001,mobilenet,20250117
1,0.259689,0.014790,0.220086,0.007877,0.001,mobilenet,20250117
2,0.150853,0.009059,0.196414,0.007532,0.001,mobilenet,20250117


In [3]:
history

,iou,loss,val_iou,val_loss,learning_rate,modelname,train_date
0,0.141935,0.055430,0.285442,0.013452,0.001,mobilenet,20250117
1,0.259689,0.014790,0.220086,0.007877,0.001,mobilenet,20250117
2,0.150853,0.009059,0.196414,0.007532,0.001,mobilenet,20250117


In [3]:
import pandas as pd
from managers.DataGeneratorSkillsTorch import DataGeneratorSkills
from managers.DataRepository import DataRepository
from managers.FrameLoader import FrameLoader

In [4]:
DIM = 224
repo = DataRepository()
train_generator = DataGeneratorSkills(
    frameloader=FrameLoader(repo),
    train_test_val="train",
    dim=(DIM,DIM),
    timesteps=16,
    batch_size=1,
)

connection established, loading frame paths
relative paths of framelabels loaded
connection established, loading frame paths
relative paths of framelabels loaded
skillcounts    skills  types  turners
0      30      6       26
skill
1     2273
3      631
2      586
4      442
7      144
5       90
15      50
11      41
10      40
20      36
12      33
17      31
13      20
25      16
14      14
18      12
23       8
30       6
8        6
19       3
24       2
9        1
26       1
Name: count, dtype: int64
90
DataGeneratorSkills init done
skill
1     2273
3      631
2      586
4      442
7      144
5       90
15      50
11      41
10      40
20      36
12      33
17      31
13      20
25      16
14      14
18      12
23       8
8        6
30       6
19       3
24       2
9        1
26       1
Name: count, dtype: int64
90


In [20]:
skillValueCounts = train_generator.Skills["skill"].value_counts()
lowestTrainAmount = min(
    skillValueCounts.loc[1], # Jumps
    skillValueCounts.loc[2], # Returns
    skillValueCounts.loc[3], # Pushups
    skillValueCounts.loc[4], # Frogs
    skillValueCounts[skillValueCounts.index >= 5].sum(),
)

df = pd.concat([
    train_generator.Skills[train_generator.Skills['skill'] == 1].iloc[:lowestTrainAmount],
    train_generator.Skills[train_generator.Skills['skill'] == 2].iloc[:lowestTrainAmount],
    train_generator.Skills[train_generator.Skills['skill'] == 3].iloc[:lowestTrainAmount],
    train_generator.Skills[train_generator.Skills['skill'] == 4].iloc[:lowestTrainAmount],
    train_generator.Skills[train_generator.Skills['skill'] >= 5].iloc[:lowestTrainAmount]
], ignore_index=True)

In [21]:
df

,id,videoId,frameStart,frameEnd,type,rotations,turner1,turner2,skill,hands,feet,turntable,bodyRotations,backwards,sloppy,hard2see,fault,labeldate
0,1705,1281,285,301,1,1,1,1,1,0,2,0,0,0,0,0,0,2025-02-26
1,1825,1277,455,478,1,1,1,1,1,0,2,0,0,0,0,0,0,2025-02-26
2,3625,2278,2265,2296,4,1,1,1,1,0,2,0,0,0,0,0,0,2025-03-29
3,3991,2308,902,933,1,1,1,1,1,0,2,0,0,0,0,0,0,2025-04-04
4,5010,2298,1365,1394,4,1,1,1,1,0,2,0,0,0,0,0,0,2025-04-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,3307,692,545,569,1,2,3,3,20,2,2,0,0,0,0,0,0,2025-03-11
2206,1140,1202,1217,1243,6,1,1,1,10,2,1,0,0,0,0,0,0,2025-02-09
2207,2160,688,1007,1023,1,1,8,1,7,1,2,0,0,0,0,0,0,2025-03-02
2208,224,2216,1964,1998,1,2,2,1,7,0,2,0,0,0,0,0,0,2025-02-09


In [19]:
len(df) / 5

442.0